## Homework

The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.
Time used: 44mins.


## Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page),
but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Download the data for January and February 2023.
Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

Answer: 19

In [50]:
import pandas as pd
import numpy as np
import copy
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from scipy.sparse import csr_matrix
from typing import List

In [3]:
path_jan = "/data/homework/yellow_tripdata_2023-01.parquet"
path_feb = "/data/homework/yellow_tripdata_2023-02.parquet"

In [55]:
df_jan = pd.read_parquet(path_jan)
df_feb = pd.read_parquet(path_feb)
df_jan.head()

In [6]:
col_count_jan = len(df_jan.columns)
print(col_count_jan)

## Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

Answer: 42.59

In [15]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime'] 
# use apply function and x.total_seconds
df_jan['duration'] = df_jan['duration'].apply(lambda x: x.total_seconds()/60)

In [18]:
print(np.std(df_jan['duration'].tolist()))

## Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

Answer: 98%

In [21]:
df_jan_og = copy.deepcopy(df_jan)

In [22]:
df_jan = df_jan[(df_jan['duration'] >= 1.0)&(df_jan['duration'] <= 60.0)]

In [23]:
print(len(df_jan)/len(df_jan_og))

## Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

Answer: 515

In [32]:
features = ['PULocationID', 'DOLocationID']
for f in features:
    df_jan[f] = df_jan[f].astype('str')

In [34]:
df_jan_dict = df_jan[features].to_dict(orient='records')

In [60]:
dv = DictVectorizer()
x_train = dv.fit_transform(df_jan_dict)

In [38]:
len(dv.feature_names_)

In [48]:
type(x_train)

## Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

Answer: 7.64

In [43]:
lin_reg=LinearRegression()
y_train = df_jan['duration'].tolist()
lin_reg.fit(x_train,y_train)

In [46]:
y_pred = lin_reg.predict(x_train)
rmse = root_mean_squared_error(y_train,y_pred)

In [47]:
rmse

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81

Answer: 7.81

In [77]:
def lebal_preprocess(df: pd.DataFrame) -> List:
    df = df.copy()
    df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime'] 
    df['duration'] = df['duration'].apply(lambda x: x.total_seconds()/60)
    df = df[(df['duration'] >= 1.0)&(df['duration'] <= 60.0)]
    return df['duration'].tolist(),df

In [75]:
def features_preprocess(df: pd.DataFrame, dv:DictVectorizer, features: List[str]=['PULocationID', 'DOLocationID']) -> csr_matrix:
    for f in features:
        df[f] = df[f].astype('str')
    df_dict = df[features].to_dict(orient='records')
    x = dv.transform(df_dict)
    return x

In [78]:
y_val,df_feb = lebal_preprocess(df_feb)
x_val = features_preprocess(df_feb,dv)

In [71]:
y_pred_val = lin_reg.predict(x_val)
rmse = root_mean_squared_error(y_val,y_pred_val)

In [73]:
print(rmse)